# 0.4.0 Clustering in Real World

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np

import matplotlib.image as img

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import warnings
warnings.filterwarnings("ignore")

In [3]:
import sys
sys.path.append("../") 

import utils.paths as path
from utils.paths2 import direcciones

## 0.4.2 Extract RGB values from image

In [5]:
# Read batman image and print dimensions
batman_image = img.imread(path.data_external_dir('batman.jpg'))
print(batman_image.shape)

(169, 269, 3)


In [6]:
r = []
g = []
b = []

# Store RGB values of all pixels in lists r, g and b
for row in batman_image:
    for temp_r, temp_g, temp_b in row:
        r.append(temp_r)
        g.append(temp_g)
        b.append(temp_b)

## 0.4.3 How many dominant colors?

In [ ]:
distortions = []
num_clusters = range(1, 7)

In [ ]:
# Create a list of distortions from the kmeans function
for i in ____:
    cluster_centers, distortion = ____
    distortions.append(____)

In [8]:
batman_image

array([[[37, 27, 25],
        [36, 22, 19],
        [48, 22, 23],
        ...,
        [33, 30, 49],
        [21, 29, 52],
        [36, 53, 71]],

       [[10,  5,  2],
        [ 9,  0,  0],
        [15,  0,  0],
        ...,
        [11,  0, 17],
        [ 4,  5, 25],
        [22, 27, 46]],

       [[ 1,  1,  0],
        [ 4,  3,  1],
        [12,  2,  3],
        ...,
        [28,  0, 12],
        [16,  0, 15],
        [37, 20, 36]],

       ...,

       [[ 0,  2,  7],
        [ 6,  7,  9],
        [ 5,  5,  3],
        ...,
        [ 6,  0,  2],
        [ 6,  2,  3],
        [27, 25, 26]],

       [[ 0,  4,  3],
        [ 1,  3,  0],
        [ 0,  1,  0],
        ...,
        [ 7,  1,  3],
        [ 6,  2,  3],
        [27, 25, 26]],

       [[26, 30, 29],
        [28, 30, 27],
        [28, 29, 21],
        ...,
        [32, 26, 28],
        [18, 14, 15],
        [51, 49, 50]]], dtype=uint8)

In [46]:
print('ok_')

ok_
